In [18]:
import mysql.connector as SQLC

DataBase = SQLC.connect(
   host ="*********",
   user ="*******",
   password ="*********",
    database ="*********"
)
Cursor = DataBase.cursor()

In [19]:
import re 
from Bio import SeqIO
from Bio import Entrez
Entrez.email = '********@alunos.uminho.pt'

In [20]:
#fonte = open("L42022.1.gb",'r')
fonte = open("L42023.1.gb",'r')
locus = fonte.read()
fonte.close()

In [21]:
record = SeqIO.read("L42023.1.gb", 'genbank')

### Tabela Locus ###

In [22]:
accession=''
accession_regex = re.search(r'ACCESSION\s+(\w+)', locus)
if accession_regex:
    accession = accession_regex.group(1)

locus_name = ""
locus_name_regex = re.match(r'LOCUS\s+(\w+)', locus)
if locus_name_regex:
    locus_name = locus_name_regex.group(1)

molecule = ''
molecule_regex = re.search(r'LOCUS[\s\w]+\d+\s[bp]{2}\s+(\w+)', locus)
if molecule_regex:
    molecule = molecule_regex.group(1)

segment_type = ''
segment_type_regex = re.search(r'LOCUS[\s\w]+\d+\s[bp]{2}\s+\w+\s+(\w+)', locus)
if segment_type_regex:
    segment_type = segment_type_regex.group(1)

genbank_date = ''
genbank_date_regex = re.search(r'LOCUS[\s\w]+\d+\s[bp]{2}[\s\w]+\w{3}\s+(\d+-[A-Z]+-\d+)', locus)
if genbank_date_regex:
    genbank_date = genbank_date_regex.group(1)

genbank_division = ''
genbank_division_regex = re.search(r'LOCUS[\s\w]+\d+\s[bp]{2}[\s\w]+(\w{3})', locus)
if genbank_division_regex:
    genbank_division = genbank_division_regex.group(1)


sql_locus = f"INSERT INTO locus (accession, locus_name, molecule, segment_type, genbank_date, genbank_division) VALUES ('{accession}', '{locus_name}', '{molecule}', '{segment_type}', '{genbank_date}', '{genbank_division}')"

print(sql_locus)

Cursor.execute(sql_locus)
DataBase.commit()



INSERT INTO locus (accession, locus_name, molecule, segment_type, genbank_date, genbank_division) VALUES ('L42023', 'L42023', 'DNA', 'circular', '31-JAN-2014', 'BCT')


### Tabela sequences ###

In [36]:
seq = ""
seq_regex = re.findall(r'^\s+\d+ [actg ]+', locus, re.MULTILINE )
if seq_regex:
    for linha in seq_regex:
        m = re.match( r'\s+\d+ (.+)', linha, re.DOTALL )
        seq = seq + re.sub(r'\s+', '', m.group(1) )

seq_length = len(seq)

locus_accession = accession

sql_sequences = f"INSERT INTO sequences (seq, seq_length, locus_accession) VALUES ('{seq.upper()}', {seq_length}, '{locus_accession}')"

print(sql_sequences)

Cursor.execute(sql_sequences)

DataBase.commit()


INSERT INTO sequences (seq, seq_length, locus_accession) VALUES ('...', 1827071, 'L42023')


### Tabela definition ###


In [24]:
definition = record.description

locus_accession = accession

sql_definition = f"INSERT INTO definition (definition, locus_accession) VALUES ('{definition}', '{locus_accession}')"

print(sql_definition)

Cursor.execute(sql_definition)

DataBase.commit()

INSERT INTO definition (definition, locus_accession) VALUES ('Haemophilus influenzae Rd KW20, complete genome', 'L42023')


### Tabela locus_keywords ###

In [25]:
keyword_dict = {}

keywords_name = record.annotations['keywords']

for num, keyword in enumerate(keywords_name):
    if keywords_name == ['']:
        print('Este ficheiro não tem keywords associadas')
    else:
        sql_locus_keywords = f"INSERT INTO locus_keywords (locus_accession, key_order, keyword_name) VALUES ('{locus_accession}', {num+1}, '{keyword}')"
        Cursor.execute(sql_locus_keywords)
        DataBase.commit()
        print(sql_locus_keywords)

Este ficheiro não tem keywords associadas


### Tabela source ###

In [26]:
scientific_name = record.annotations['source']

sql_source = f"INSERT INTO source (scientific_name, locus_accession) VALUES ('{scientific_name}', '{locus_accession}')"

print(sql_source)

Cursor.execute(sql_source)

DataBase.commit()

INSERT INTO source (scientific_name, locus_accession) VALUES ('Haemophilus influenzae Rd KW20', 'L42023')


### Tabela taxonomy ###


In [27]:
taxonomy = record.annotations['taxonomy']

sql_taxonomy = f"INSERT INTO taxonomy (taxonomy, locus_accession) VALUES ('{'; '.join(taxonomy)}', '{locus_accession}')"

print(sql_taxonomy)

Cursor.execute(sql_taxonomy)

DataBase.commit()

INSERT INTO taxonomy (taxonomy, locus_accession) VALUES ('Bacteria; Proteobacteria; Gammaproteobacteria; Pasteurellales; Pasteurellaceae; Haemophilus', 'L42023')


### Tabela reference ###

In [28]:
title = list()
journal = list()
pubmed_id = list()
ref_location = list()


for reference in record.annotations['references']:
    title.append(reference.title)
    journal.append(reference.journal)
    pubmed_id.append(reference.pubmed_id)
    for loc in reference.location:
        ref_location.append(loc)

    
for i in zip(title, journal, pubmed_id, ref_location):
    if i[2] == '' and i[3] == '':
        sql_references = f"INSERT INTO reference (title, journal, locus_accession) VALUES ('{i[0]}', '{i[1]}', '{locus_accession}')"
        print(sql_references)
        print('Esta referência não possui ID do pubmed, nem descreve a sua localização')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()

    elif i[2] == '' and i[3] != '':
        sql_references = f"INSERT INTO reference (title, journal, locus_accession, ref_location) VALUES ('{i[0]}', '{i[1]}', '{locus_accession}', '{i[3]}')"
        print(sql_references)
        print('Esta referência não possui ID do pubmed')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()

    elif i[2] != '' and i[3] == '':
        sql_references = f"INSERT INTO reference (title, journal, pubmed_id, locus_accession) VALUES ('{i[0]}', '{i[1]}', '{i[2]}', '{locus_accession}')"
        print(sql_references)
        print('Esta referência não descreve a sua localização')
        print()
        Cursor.execute(sql_references)
        DataBase.commit()
        
    else:
        sql_references = f"INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ('{i[0]}', '{i[1]}', '{i[2]}', '{locus_accession}', '{i[3]}')"
        print(sql_references)
        print()
        Cursor.execute(sql_references)
        DataBase.commit()
        

INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ('Whole-genome random sequencing and assembly of Haemophilus influenzae Rd', 'Science 269 (5223), 496-512 (1995)', '7542800', 'L42023', '[0:1830138]')

INSERT INTO reference (title, journal, pubmed_id, locus_accession, ref_location) VALUES ('Metabolism and evolution of Haemophilus influenzae deduced from a whole-genome comparison with Escherichia coli', 'Curr. Biol. 6 (3), 279-291 (1996)', '8805245', 'L42023', '[0:1830138]')

INSERT INTO reference (title, journal, locus_accession, ref_location) VALUES ('Direct Submission', 'Submitted (25-JUL-1995) The Institute for Genomic Research, 9712 Medical Center Dr, Rockville, MD 20850, USA', 'L42023', '[0:1830138]')
Esta referência não possui ID do pubmed

INSERT INTO reference (title, journal, locus_accession, ref_location) VALUES ('Direct Submission', 'Submitted (27-SEP-1997) The Institute for Genomic Research, 9712 Medical Center Dr, Rockville, MD 20850, 

### Tabela pubmed ###

In [29]:
for id in pubmed_id:
    if id != '':
        pubmed_handle = Entrez.esummary(db="pubmed", id=id, retmode="xml")
        pubmed_record = Entrez.read(pubmed_handle)
        pubmed_handle.close()
        for info in pubmed_record:
            title_pubmed = list()
            pub_date = list()
            pub_source = list()
            pub_issue = list()
            pub_pages = list()
            doi = list()

            title_pubmed.append(info['Title'])
            pub_date.append(info['PubDate'])
            pub_source.append(info['Source'])
            pub_issue.append(info['Issue'])
            pub_pages.append(info['Pages'])
            doi.append(info['DOI'])
        
        for i in zip(title_pubmed, pub_date, pub_source, pub_issue, pub_pages, doi):
            sql_pubmed = f"INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages, doi) VALUES ({id}, '{i[0]}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}', '{i[5]}')"    
            Cursor.execute(sql_pubmed)
            DataBase.commit()
            print(sql_pubmed)



INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages, doi) VALUES (7542800, 'Whole-genome random sequencing and assembly of Haemophilus influenzae Rd.', '1995 Jul 28', 'Science', '5223', '496-512', '10.1126/science.7542800')
INSERT INTO pubmed (pubmed_id, title, date, source, issue, pages, doi) VALUES (8805245, 'Metabolism and evolution of Haemophilus influenzae deduced from a whole-genome comparison with Escherichia coli.', '1996 Mar 1', 'Curr Biol', '3', '279-91', '10.1016/s0960-9822(02)00478-5')


### Tabela authors ###

In [30]:
author_names = dict()

for id in pubmed_id:
    if id != '':
        pubmed_handle = Entrez.esummary(db="pubmed", id=id, retmode="xml")
        pubmed_record = Entrez.read(pubmed_handle)
        pubmed_handle.close()

        for info in pubmed_record:
            author_names[id] = info['AuthorList']
            
        for id, authors in author_names.items():
            for name in authors:
                sql_authors = f"INSERT INTO authors (author_name, pubmed_id) VALUES ('{name}', {id})"
                print(sql_authors)
                Cursor.execute(sql_authors)
                DataBase.commit()
               



INSERT INTO authors (author_name, pubmed_id) VALUES ('Fleischmann RD', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Adams MD', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('White O', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Clayton RA', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Kirkness EF', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Kerlavage AR', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Bult CJ', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Tomb JF', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Dougherty BA', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Merrick JM', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Fleischmann RD', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('Adams MD', 7542800)
INSERT INTO authors (author_name, pubmed_id) VALUES ('White O', 7542800)
INSERT INTO aut

### Tabela feat_source ###

In [31]:
features = record.features

for i in features:
    if i.type == 'source':
        location = set(i for i in i.location)
        organism = set(i for i in i.qualifiers['organism'])
        db_xref = set(i.qualifiers['db_xref'])

        sql_feat_source = f'INSERT INTO feat_source (location, organism, db_xref, locus_accession) VALUES ("{i.location}", "{organism}", "{db_xref}", "{locus_accession}")'
        print(sql_feat_source)
        Cursor.execute(sql_feat_source)
        DataBase.commit()
        


INSERT INTO feat_source (location, organism, db_xref, locus_accession) VALUES ("[0:1830138](+)", "{'Haemophilus influenzae Rd KW20'}", "{'taxon:71421'}", "L42023")


### Tabela feat_cds ###

In [32]:
location_list = list()
codon_start_list = list()
product_list = list()
protein_id_list = list()
translation_list = list()

for i in features:
    if i.type == 'CDS':
            
        location_list.append(i.location)
        codon_start_list.append(i.qualifiers['codon_start'])
        protein_id_list.append(i.qualifiers['protein_id'])
        translation_list.append(i.qualifiers['translation'])
        if 'product' in i.qualifiers.keys():
            product_list.append(i.qualifiers['product'])

if 'product' in i.qualifiers.keys():
   for i in zip(location_list, codon_start_list, product_list, protein_id_list, translation_list):
        sql_feat_cds = f'INSERT INTO feat_cds (location, codon_start, product, protein_ib, translation, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{i[2][0]}", "{i[3][0]}", "{i[4][0]}", "{locus_accession}")'
        print(sql_feat_cds) 
        Cursor.execute(sql_feat_cds)
        DataBase.commit()
        
else:
    for i in zip(location_list, codon_start_list, protein_id_list, translation_list):
        sql_feat_cds = f'INSERT INTO feat_cds (location, codon_start, protein_ib, translation, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{i[2][0]}", "{i[3][0]}", "{locus_accession}")'
        print(sql_feat_cds)
        Cursor.execute(sql_feat_cds)
        DataBase.commit()
        

INSERT INTO feat_cds (location, codon_start, product, protein_ib, translation, locus_accession) VALUES ("[1:1021](+)", "1", "glyceraldehyde-3-phosphate dehydrogenase (gapdH)", "AAC21680.1", "MAIKIGINGFGRIGRIVFRAAQHRDDIEVVGINDLIDVEYMAYMLKYDSTHGRFDGTVEVKDGNLVVNGKTIRVTAERDPANLNWGAIGVDIAVEATGLFLTDETARKHITAGAKKVVLTGPSKDATPMFVRGVNFNAYAGQDIVSNASCTTNCLAPLARVVHETFGIKDGLMTTVHATTATQKTVDGPSAKDWRGGRGASQNIIPSSTGAAKAVGKVLPALNGKLTGMAFRVPTPNVSVVDLTVNLEKPASYDAIKQAIKDAAEGKTFNGELKGVLGYTEDAVVSTDFNGCALTSVFDADAGIALTDSFVKLVSWYDNETGYSNKVLDLVAHIYNYKG", "L42023")
INSERT INTO feat_cds (location, codon_start, product, protein_ib, translation, locus_accession) VALUES ("[1189:3013](+)", "1", "long chain fatty acid coA ligase, putative", "AAC21681.1", "MKIKNRVNMNLDLHFVHRIQQQAKTRTNMTALRYKEHGLWRDISWKNFQEQLNQLSRALLAHNIDVQDKIAIFAHNMERWTIVDIATLQIRAITVPIYATNTAQQAEFILNHADVKILFVGDQEQYDQTLEIAHHCPKLQKIVAMKSTIQLQQDPLSCTWESFIKTGSNAQQDELTQRLNQKQLSDLFTIIYTSGTTGEPKGVMLDYANLAHQLETHDLSLNVTDQDISLSFLPFSHIFERAWAAYILHRGAILCYLEDTNQVRSALTE

### Tabela feat_gene ###

In [33]:
for i in features:
    if i.type == 'gene':
        
        gene_location_list = list()
        gene_name_list = list()

        gene_location_list.append(i.location)
        
        if 'locus_tag' in i.qualifiers.keys():
            gene_name_list.append(i.qualifiers['locus_tag'])
        elif 'gene' in i.qualifiers.keys():
            gene_name_list.append(i.qualifiers['gene'])

        if 'locus_tag' or 'gene' in i.qualifiers.keys():
            for i in zip(gene_location_list, gene_name_list):
                sql_feat_gene = f'INSERT INTO feat_gene (location, name, locus_accession) VALUES ("{i[0]}", "{i[1][0]}", "{locus_accession}")'
                print(sql_feat_gene)    
                Cursor.execute(sql_feat_gene)
                DataBase.commit()
                
        else:
            for i in gene_location_list:
                sql_feat_gene = f'INSERT INTO feat_gene (location, locus_accession) VALUES ("{i}", "{locus_accession}")'
                print(sql_feat_gene)    
                Cursor.execute(sql_feat_gene)
                DataBase.commit()
                

INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[1:1021](+)", "HI_0001", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[1189:3013](+)", "HI_0002", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[3049:3838](-)", "HI_0003", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[3853:4318](-)", "HI_0004", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[4578:5391](-)", "HI_0005", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[5655:8748](+)", "HI_0006", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[8749:9688](+)", "HI_0007", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[9680:10397](+)", "HI_0008", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[10466:11375](+)", "HI_0009", "L42023")
INSERT INTO feat_gene (location, name, locus_accession) VALUES ("[11413:1